In [1]:
from pprint import pprint
import pandas as pd
import numpy as np
import gc
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score


In [2]:
# Data Directory
data_dir = 'dataset/'

# Load the data
train_1 = pd.read_csv(data_dir + 'train_1.csv')
train_2_1 = pd.read_csv(data_dir + 'train_2_1.csv')
train_2_2 = pd.read_csv(data_dir + 'train_2_2.csv')
test_1 = pd.read_csv(data_dir + 'test_1.csv')
test_2_1 = pd.read_csv(data_dir + 'test_2_1.csv')
test_2_2 = pd.read_csv(data_dir + 'test_2_2.csv')

/tmp/ipykernel_30195/3751997310.py:5: DtypeWarning: Columns (126,128,143) have mixed types. Specify dtype option on import or set low_memory=False.
  train_1 = pd.read_csv(data_dir + 'train_1.csv')
/tmp/ipykernel_30195/3751997310.py:6: DtypeWarning: Columns (675,676,677) have mixed types. Specify dtype option on import or set low_memory=False.
  train_2_1 = pd.read_csv(data_dir + 'train_2_1.csv')
/tmp/ipykernel_30195/3751997310.py:7: DtypeWarning: Columns (675,676,677) have mixed types. Specify dtype option on import or set low_memory=False.
  train_2_2 = pd.read_csv(data_dir + 'train_2_2.csv')


In [3]:
# Remove duplicates based on 'id' in the additional information DataFrames
train_2_1 = train_2_1.drop_duplicates(subset='id')
train_2_2 = train_2_2.drop_duplicates(subset='id')
test_2_1 = test_2_1.drop_duplicates(subset='id')
test_2_2 = test_2_2.drop_duplicates(subset='id')

# Merge the DataFrames
train = pd.merge(train_1, train_2_1, on='id', how='left')
train = pd.merge(train, train_2_2, on='id', how='left')

test = pd.merge(test_1, test_2_1, on='id', how='left')
test = pd.merge(test, test_2_2, on='id', how='left')

# Replace "NR" with NaN
train.replace("NR", np.nan, inplace=True)
test.replace("NR", np.nan, inplace=True)

In [4]:
from scipy import stats

# Identify numerical and categorical columns
numerical_cols_train = train.select_dtypes(include=['int64', 'float64']).columns
categorical_cols_train = train.select_dtypes(include=['object']).columns

numerical_cols_test = test.select_dtypes(include=['int64', 'float64']).columns
categorical_cols_test = test.select_dtypes(include=['object']).columns

In [5]:
#convert col to numeric
def convert_to_numeric(df, col):
    try:
        df[col] = pd.to_numeric(df[col])
    except ValueError:
        pass

for col in categorical_cols_train:
    convert_to_numeric(train, col)
for col in categorical_cols_test:
    convert_to_numeric(test, col)  

In [6]:
# # Function to print specific data types in a column
# def print_data_types(column):
#     data_types = column.apply(lambda x: type(x).__name__)
#     print(data_types.value_counts())

# print("Data types in 'add_671_x':")
# print_data_types(train['add_671_x'])

# # Filter only float type values from the column
# float_values = train['add_671_x'].apply(lambda x: isinstance(x, float))

# # Get the filtered DataFrame
# filtered_df = train['add_671_x'][float_values]

In [7]:
# Convert string to list
import ast

def convert_to_list(value):
    if pd.isna(value):
        return np.nan
    try:
        value = value.strip('"')
        return ast.literal_eval(value)
    except (ValueError, SyntaxError):
        return np.nan
def handle_NaN(value):
    #check if value is float
    if isinstance(value, float):
        return []
    return value
for col in ['add_671_x', 'add_671_y']:
    train[col] = train[col].apply(convert_to_list)
    test[col] = test[col].apply(convert_to_list)
for col in ['add_671_x', 'add_671_y']:
    train[col] = train[col].apply(handle_NaN)
    test[col] = test[col].apply(handle_NaN)


In [8]:
# col = 'add_671_x'
# max_length = train[col].apply(len).max()
# for i in range(max_length):
#     k = train[col].apply(lambda x: x[i] if len(x) > i else np.nan)
#     print(k)


In [9]:
def flatten_lists(df1,df2, col):
    max_length = df1[col].apply(len).max()
    max_length = max(max_length, df2[col].apply(len).max())
    for i in range(max_length):
        name = col + "_" + str(i)
        df1[name] = df1[col].apply(lambda x: x[i] if i < len(x) else np.nan)
        convert_to_numeric(df1, name)
        df2[name] = df2[col].apply(lambda x: x[i] if i < len(x) else np.nan)
        convert_to_numeric(df2, name)
        
    df1.drop(columns=[col], inplace=True)
    df2.drop(columns=[col], inplace=True)
    return max_length
flatten_lists(train, test, 'add_671_x')
flatten_lists(train, test, 'add_671_y')


np.int64(5)

In [10]:
# Identify numerical and categorical columns
numerical_cols_train = train.select_dtypes(include=['int64', 'float64']).columns
categorical_cols_train = train.select_dtypes(include=['object']).columns

numerical_cols_test = test.select_dtypes(include=['int64', 'float64']).columns
categorical_cols_test = test.select_dtypes(include=['object']).columns


print('Categorical columns train: ', categorical_cols_train)
print('Categorical columns test: ', categorical_cols_test)
# Print how many distinct values each categorical column has
print("\nDistinct values in categorical columns (train):")
for col in categorical_cols_train:
    print(f"{col}: {train[col].nunique()}")

print("\nDistinct values in categorical columns (test):")
for col in categorical_cols_test:
    print(f"{col}: {test[col].nunique()}")


Categorical columns train:  Index(['loan_id', 'id', 'prod', 'col_114', 'col_117', 'col_118', 'col_123',
       'col_140', 'col_148', 'col_150', 'col_156', 'col_157', 'col_158',
       'col_161', 'add_431_x', 'add_672_x', 'add_673_x', 'add_675_x',
       'add_676_x', 'add_677_x', 'add_431_y', 'add_672_y', 'add_673_y',
       'add_675_y', 'add_676_y', 'add_677_y'],
      dtype='object')
Categorical columns test:  Index(['loan_id', 'id', 'prod', 'col_114', 'col_117', 'col_118', 'col_123',
       'col_140', 'col_148', 'col_150', 'col_156', 'col_157', 'col_158',
       'col_161', 'add_431_x', 'add_672_x', 'add_673_x', 'add_675_x',
       'add_676_x', 'add_677_x', 'add_431_y', 'add_672_y', 'add_673_y',
       'add_675_y', 'add_676_y', 'add_677_y'],
      dtype='object')

Distinct values in categorical columns (train):
loan_id: 100000
id: 99584
prod: 5
col_114: 26
col_117: 2
col_118: 7
col_123: 3
col_140: 20
col_148: 1
col_150: 4
col_156: 9
col_157: 6
col_158: 7
col_161: 5
add_431_x: 2
add_67

In [11]:
# Handle missing values for numerical columns train
for col in numerical_cols_train:
    train[col] = train[col].fillna(train[col].mean())
# Handle missing values for categorical columns train
for col in categorical_cols_train:
    train[col] = train[col].fillna('missing')

# Handle missing values for numerical columns test
for col in numerical_cols_test:
    test[col] = test[col].fillna(test[col].mean())
# Handle missing values for categorical columns test
for col in categorical_cols_test:
    test[col] = test[col].fillna('missing')

# # Handle outliers for numerical columns using Z-score
# for col in numerical_cols_train:
#     train = train[(np.abs(stats.zscore(train[col])) < 3)]
# for col in numerical_cols_test:
#     test = test[(np.abs(stats.zscore(test[col])) < 3)]

In [12]:
# Release memory by deleting unnecessary variables and forcing garbage collection
del  test_2_1, test_2_2, train_1, train_2_1, train_2_2
gc.collect()

0

In [13]:
# Select relevant features
# features = train.columns.drop(categorical_cols_train).drop('label')
features = train.columns.drop(['id', 'label','loan_id'])
X_train = train[features]
y_train = train['label']
X_test = test[features]

In [ ]:
# Release memory by deleting unnecessary variables and forcing garbage collection
del train, test
gc.collect()

In [14]:
# Remove 'loan_id' and 'id' from the list of categorical columns if they exist
categorical_cols_train = [col for col in categorical_cols_train if col not in ['loan_id', 'id']]
categorical_cols_test = [col for col in categorical_cols_test if col not in ['loan_id', 'id']]

# Remove 'loan_id' and 'id' from the features if they exist
X_train = X_train.drop(columns=['loan_id', 'id'], errors='ignore')
X_test = X_test.drop(columns=['loan_id', 'id'], errors='ignore')

# Convert categorical columns to numerical using one-hot encoding
X_train = pd.get_dummies(X_train, columns=categorical_cols_train)
X_test = pd.get_dummies(X_test, columns=categorical_cols_test)

# Ensure the same columns in train and test after one-hot encoding
X_train, X_test = X_train.align(X_test, join='inner', axis=1, fill_value=0)

# Rename columns to ensure they are valid strings without special characters
X_train.columns = [str(col).replace('[', '').replace(']', '').replace('<', '').replace('>', '') for col in X_train.columns]
X_test.columns = [str(col).replace('[', '').replace(']', '').replace('<', '').replace('>', '') for col in X_test.columns]



In [15]:

import xgboost as xgb
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import roc_auc_score

# Split data for validation
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

# Train the XGBoost model
model = xgb.XGBClassifier(
    n_estimators=100,        # Increase the number of boosting rounds
    learning_rate=0.1,       # Reduce the learning rate
    max_depth=6,             # Increase the maximum depth of trees
    eval_metric='auc',        # Evaluation metric
    random_state=42           # Seed for reproducibility
)
model.fit(
    X_train, y_train,
    eval_set=[(X_val, y_val)],
    verbose=True
    )


[0]	validation_0-auc:0.82390
[1]	validation_0-auc:0.82632
[2]	validation_0-auc:0.83652
[3]	validation_0-auc:0.84026
[4]	validation_0-auc:0.84253
[5]	validation_0-auc:0.84375
[6]	validation_0-auc:0.84593
[7]	validation_0-auc:0.84903
[8]	validation_0-auc:0.85341
[9]	validation_0-auc:0.85346
[10]	validation_0-auc:0.85520
[11]	validation_0-auc:0.85600
[12]	validation_0-auc:0.85699
[13]	validation_0-auc:0.85914
[14]	validation_0-auc:0.85927
[15]	validation_0-auc:0.86002
[16]	validation_0-auc:0.86067
[17]	validation_0-auc:0.86134
[18]	validation_0-auc:0.86189
[19]	validation_0-auc:0.86306
[20]	validation_0-auc:0.86324
[21]	validation_0-auc:0.86385
[22]	validation_0-auc:0.86540
[23]	validation_0-auc:0.86526
[24]	validation_0-auc:0.86617
[25]	validation_0-auc:0.86648
[26]	validation_0-auc:0.86650
[27]	validation_0-auc:0.86800
[28]	validation_0-auc:0.86841
[29]	validation_0-auc:0.86941
[30]	validation_0-auc:0.87038
[31]	validation_0-auc:0.87143
[32]	validation_0-auc:0.87164
[33]	validation_0-au

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='auc', feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.1, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=6, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=100, n_jobs=None,
              num_parallel_tree=None, random_state=42, ...)

In [16]:
# You can then make predictions on the test set
predictions = model.predict_proba(X_test)[:, 1]  # Get probabilities for class 1

# Ensure the lengths match
# predictions = predictions[:len(test_1)]

# Save predictions to a submission file
submission = pd.DataFrame({
    'loan_id': test_1['loan_id'],
    'prob': predictions
})
submission.to_csv('submission.csv', index=False)